In [1]:
# Imports

import pandas as pd
import numpy as np

# Processsamento de strings...

import string
import re

# Remover Warnings

import warnings

warnings.filterwarnings("ignore")

In [2]:
# Colentando dados de steam_app_data.csv

df1 = pd.read_csv("steam_app_data.csv")
df1 = df1.set_index("steam_appid")
del df1.index.name

print("steam_app_data.csv - Número de linhas = {0} - Número de colunas = {1}".format(len(df1), len(df1.columns)))


steam_app_data.csv - Número de linhas = 29235 - Número de colunas = 38


In [3]:
# Colentando dados de steamspy_data.csv

df2 = pd.read_csv("steamspy_data.csv")
df2 = df2.set_index("appid")
del df2.index.name

print("steamspy_data.csv - Número de linhas = {0} - Número de colunas = {1}".format(len(df2), len(df2.columns)))

steamspy_data.csv - Número de linhas = 29235 - Número de colunas = 19


In [4]:
# Juntando tabelas

# Removendo coluna duplicada
df2 = df2.drop(columns="name")

data = df1.join(df2)

print("Dados conjutos, Número de linhas = {0} - Número de colunas = {1}".format(len(data), len(data.columns)))

# Somente jogos
data = data[data.type == "game"]
data = data.drop(columns="type")

# Modifica o index para o nome do jogo
data = data.set_index("name")

# Deixar tabela toda em minusculo
data = data.apply(lambda x: x.astype(str).str.lower());

Dados conjutos, Número de linhas = 29235 - Número de colunas = 56


In [5]:
data.head(1)

,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,...,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,0.0,false,nan,nan,play the world's number 1 online action game. ...,play the world's number 1 online action game. ...,play the world's number 1 online action game. ...,nan,"english<strong>*</strong>, french<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,...,709.0,317.0,26.0,999.0,999.0,0.0,"english, french, german, italian, spanish - sp...",action,14923.0,"{'action': 2681, 'fps': 2048, 'multiplayer': 1..."


In [6]:
# Processa as linguagens suportadas pelos jogos...

texto_conjunto = data["supported_languages"].str.cat(sep=" ") + " " + data["languages"].str.cat(sep=" ")

# Remove pontuções + bb code
texto_conjunto = re.sub("[" + string.punctuation + "]", "", texto_conjunto)
texto_conjunto = re.sub("b[br]", "", texto_conjunto)

# Arruma "brazil"
texto_conjunto = texto_conjunto.replace("azil", "brazil")

# Separadores para espaço
texto_conjunto = texto_conjunto.replace(",", " ").replace("languages", " ").replace("lang", " ")

# Remove palavras que não sao linguas do texto
texto_conjunto = texto_conjunto.replace("support", "").replace("strong", "").replace("only", "")
texto_conjunto = texto_conjunto.replace("ed", "").replace("fullaudio", "").replace("all", "").replace("full", "")
texto_conjunto = texto_conjunto.replace("finnish", "").replace("interface", "").replace("not", "")
texto_conjunto = texto_conjunto.replace("audio", "").replace("nan", "").replace("text", "").replace("with", "")

# Remove linguas juntadas sem espaço (ja possui as mesmas com o espaço)
texto_conjunto = texto_conjunto.replace("hungarianpolish", "").replace("englishgermanspanish", "")

# Arrumar simplificado (chines simplificado)
texto_conjunto = texto_conjunto.replace("simplifi", "simplified")

# Captura a lista final das liguages possiveis
liguages = set(texto_conjunto.split())

# Criar os dummy para a linguagem

i = 1

for x in liguages:
    print(i, x)
    i += 1
    data[x] = np.where(data["supported_languages"].str.contains(x) | data["languages"].str.contains(x), 1, 0)

# Remove as colunas de linguagem
data = data.drop(columns=["supported_languages", "languages"])

1 hungarian
2 chinese
3 brazil
4 latin
5 polish
6 swish
7 slovakian
8 america
9 english
10 français
11 traditional
12 czech
13 spain
14 greek
15 russian
16 italian
17 danish
18 thai
19 dutch
20 romanian
21 vietnamese
22 french
23 japanese
24 spanish
25 turkish
26 norwegian
27 german
28 korean
29 arabic
30 simplified
31 ukrainian
32 portuguese
33 bulgarian


In [7]:
data.head(1)

,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,header_image,website,...,spanish,turkish,norwegian,german,korean,arabic,simplified,ukrainian,portuguese,bulgarian
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,0.0,false,nan,nan,play the world's number 1 online action game. ...,play the world's number 1 online action game. ...,play the world's number 1 online action game. ...,nan,https://steamcdn-a.akamaihd.net/steam/apps/10/...,nan,...,1,0,0,1,1,0,1,0,0,0


In [9]:
# Atualiza os valores se o jogo é gratis

data["is_free"] = np.where(data["is_free"] == "false", 0, 1)

In [11]:
# Atualiza os valores 

data["controller_support"] = np.where(data["controller_support"] == "nan", 0, 1)

In [12]:
data.head(1)

,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,header_image,website,...,spanish,turkish,norwegian,german,korean,arabic,simplified,ukrainian,portuguese,bulgarian
name,,,,,,,,,,,,,,,,,,,,,
Counter-Strike,0.0,0,0,nan,play the world's number 1 online action game. ...,play the world's number 1 online action game. ...,play the world's number 1 online action game. ...,nan,https://steamcdn-a.akamaihd.net/steam/apps/10/...,nan,...,1,0,0,1,1,0,1,0,0,0


In [8]:
data.columns

Index(['required_age', 'is_free', 'controller_support', 'dlc',
       'detailed_description', 'about_the_game', 'short_description',
       'fullgame', 'header_image', 'website', 'pc_requirements',
       'mac_requirements', 'linux_requirements', 'legal_notice', 'drm_notice',
       'ext_user_account_notice', 'developers', 'publishers', 'demos',
       'price_overview', 'packages', 'package_groups', 'platforms',
       'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
       'movies', 'recommendations', 'achievements', 'release_date',
       'support_info', 'background', 'content_descriptors', 'developer',
       'publisher', 'score_rank', 'positive', 'negative', 'userscore',
       'owners', 'average_forever', 'average_2weeks', 'median_forever',
       'median_2weeks', 'price', 'initialprice', 'discount', 'genre', 'ccu',
       'tags', 'hungarian', 'chinese', 'brazil', 'latin', 'polish', 'swish',
       'slovakian', 'america', 'english', 'français', 'traditional', 'czech

In [17]:
data.describe()

,required_age,fullgame,score_rank,positive,negative,userscore,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
count,29086.000000,0.0,58.000000,2.923300e+04,29233.000000,29233.000000,29233.000000,29233.000000,29233.000000,29233.000000,29204.000000,29211.000000,29211.000000,29233.000000
mean,0.415011,NaN,97.034483,9.659908e+02,205.891869,0.148873,148.161906,7.163890,145.037594,7.733931,777.727743,803.913937,2.832221,140.970718
std,10.921114,NaN,2.034488,1.829953e+04,4132.530051,3.440071,1913.495513,113.656875,2389.552109,124.515254,1135.271691,1169.814839,13.353443,6894.814634
min,0.000000,NaN,93.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,NaN,96.000000,6.000000e+00,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,199.000000,199.000000,0.000000,0.000000
50%,0.000000,NaN,97.000000,2.300000e+01,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,499.000000,499.000000,0.000000,0.000000
75%,0.000000,NaN,98.750000,1.200000e+02,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,999.000000,999.000000,0.000000,0.000000
max,1818.000000,NaN,100.000000,2.644404e+06,487076.000000,100.000000,190625.000000,9003.000000,190625.000000,9003.000000,59599.000000,59599.000000,90.000000,742725.000000


In [7]:
# Limpando dados

# Somente jogos
data = data[data.type == "game"]

# # Colunas contendo dados não uteis para a regressão linear
# columns = [
#     "type",
#     "detailed_description",
#     "about_the_game",
#     "short_description",
#     "header_image",
#     "website",
#     "pc_requirements",
#     "mac_requirements",
#     "linux_requirements",
#     "legal_notice",
#     "drm_notice",
#     "demos",
#     "packages",
#     "package_groups",
#     "ext_user_account_notice",
#     "screenshots",
#     "movies"
# ]

data = data.set_index("name")
data = data.drop(columns="type")

In [11]:
# Salvando os dados

data = data.drop(columns="header_image")

data.to_excel("dados.xlsx")
del data.index.name

IllegalCharacterError: 